In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [2]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
df_train.set_index(key, inplace=True)
df_test.set_index(key, inplace=True)
train_test = pd.concat([df_train, df_test], axis=0)
# sys.exit()
df_hist = utils.read_df_pkl('../input/hist_clean_rdm0*')
df_hist = utils.reduce_mem_usage(df_hist)

df_org_lag = utils.read_pkl_gzip('../input/train_test_original_month_lag.gz')
df_org_lag = utils.reduce_mem_usage(df_org_lag)
df_org_lag.head()

df_org_lag.rename(columns={'purchase_date':'yyyymm'}, inplace=True)
print(df_hist.shape)
df_hist = df_hist.merge(df_org_lag.reset_index()[[key, 'yyyymm', 'org_month_lag']], how='inner', on=[key, 'yyyymm'])
print(df_hist.shape)

100%|██████████| 3/3 [00:11<00:00,  3.80s/it]


Mem. usage decreased to 2554.26 Mb (0.0% reduction)
Mem. usage decreased to 72.58 Mb (21.9% reduction)
(29112361, 21)
(29112361, 22)


In [3]:
# term
term_num = 10
base_list = np.zeros(term_num)
base_list = np.arange(1, term_num, 1)
last_list = np.arange(3, term_num, 1)
# for base_month, last_month in zip(base_list, last_list):

base_month = 0
last_month = 5
col_month = 'org_month_lag'
df = df_hist[df_hist[col_month]<=last_month]
df = df[df[col_month]>=base_month]

auth1 = df[df.authorized_flag==1]
auth0 = df[df.authorized_flag==0]

cat1_0 = False
# cat1_0 = True
if cat1_0:
    auth1_cat1 = auth1[auth1.category_1==1]
    auth1_cat0 = auth1[auth1.category_1==0]
    auth0_cat1 = auth0[auth0.category_1==1]
    auth0_cat0 = auth0[auth0.category_1==0]
#     del auth1, auth0
#     gc.collect()

In [4]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

for df in [auth1, auth0]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
#     df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['date_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)
    
auth1.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)
auth0.drop(['city_id', 'category_1', 'installments', 'category_2', 'category_3', 'merchant_category_id', 'merchant_id', 'purchase_amount'], axis=1, inplace=True)
auth1.head()

,card_id,purchase_date,authorized_flag,month_lag,state_id,subsector_id,first_active_month,all_term,yyyymm,yyyymmdd,weekofyear,yyyy_week,purchase_amount_over,org_month_lag,date_diff
426,C_ID_5037ff576e,2017-03-05 14:57:51,1,-9,15,33,2017-01-01,485,2017-03,2017-03-05,9,2017-09,0.0,2,690
427,C_ID_5037ff576e,2017-03-26 20:00:31,1,-9,17,33,2017-01-01,485,2017-03,2017-03-26,12,2017-12,0.0,2,669
428,C_ID_5037ff576e,2017-03-09 18:32:05,1,-9,15,19,2017-01-01,485,2017-03,2017-03-09,10,2017-10,0.0,2,686
429,C_ID_5037ff576e,2017-03-01 20:53:15,1,-9,16,34,2017-01-01,485,2017-03,2017-03-01,9,2017-09,0.0,2,694
430,C_ID_5037ff576e,2017-03-01 22:14:27,1,-9,15,33,2017-01-01,485,2017-03,2017-03-01,9,2017-09,0.0,2,694


### 日付は関係なく、直近何回、初回から何回の購買におけるデータを集計する
first 5, 10, 15, 20, 25, 30 トランザクションにおけるdate_diffを集計する  
last 5, 10, 15, 20, 25, 30 トランザクションにおけるdate_diffを集計する  
同日購買も重複削除せずに集計するversion

In [5]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([df_train, df_test], axis=0)
train_test.set_index(key, inplace=True)
aggs = {}
aggs['date_diff'] = ['mean', 'std']

df_list = [auth0, auth1]
fname_list = ['auth0', 'auth1']


for df, hist_new_name in zip(df_list, fname_list):

    df = df[[key, 'purchase_date', 'date_diff', 'yyyymm']]

    for i in tqdm(range(5, 26, 5)):
        
        df.sort_values(by=[key, 'purchase_date'], inplace=True)
        df = utils.row_number(df=df, level=key)
    
        feat_n = f'{hist_new_name}_first'
        first = df.query(f"row_no<={i}")
        tmp = first.groupby(key)['date_diff'].agg(aggs)
        new_cols = get_new_columns(f'{feat_n}{i}', aggs)
        tmp.columns = new_cols
        train_test = train_test.join(tmp)
        
        
    for i in tqdm(range(5, 26, 5)):
        
        df.sort_values(by=[key, 'purchase_date'], ascending=False, inplace=True)
        df = utils.row_number(df=df, level=key)
        
        feat_n = f'{hist_new_name}_last'
        last = df.query(f"row_no<={i}")
        tmp = last.groupby(key)['date_diff'].agg(aggs)
        new_cols = get_new_columns(f'{feat_n}{i}', aggs)
        tmp.columns = new_cols
        train_test = train_test.join(tmp)
    
train_test.head()

100%|██████████| 3/3 [00:00<00:00, 189.23it/s]


,feature_1,feature_2,feature_3,first_active_month,target,auth0_first5_date_diff_mean,auth0_first5_date_diff_std,auth0_first10_date_diff_mean,auth0_first10_date_diff_std,auth0_first15_date_diff_mean,auth0_first15_date_diff_std,auth0_first20_date_diff_mean,auth0_first20_date_diff_std,auth0_first25_date_diff_mean,auth0_first25_date_diff_std,auth0_last5_date_diff_mean,auth0_last5_date_diff_std,auth0_last10_date_diff_mean,auth0_last10_date_diff_std,auth0_last15_date_diff_mean,auth0_last15_date_diff_std,auth0_last20_date_diff_mean,auth0_last20_date_diff_std,auth0_last25_date_diff_mean,auth0_last25_date_diff_std,auth1_first5_date_diff_mean,auth1_first5_date_diff_std,auth1_first10_date_diff_mean,auth1_first10_date_diff_std,auth1_first15_date_diff_mean,auth1_first15_date_diff_std,auth1_first20_date_diff_mean,auth1_first20_date_diff_std,auth1_first25_date_diff_mean,auth1_first25_date_diff_std,auth1_last5_date_diff_mean,auth1_last5_date_diff_std,auth1_last10_date_diff_mean,auth1_last10_date_diff_std,auth1_last15_date_diff_mean,auth1_last15_date_diff_std,auth1_last20_date_diff_mean,auth1_last20_date_diff_std,auth1_last25_date_diff_mean,auth1_last25_date_diff_std
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,535.6,13.011533,502.3,37.279872,497.181818,39.229627,497.181818,39.229627,497.181818,39.229627,462.0,14.849242,491.1,35.466573,497.181818,39.229627,497.181818,39.229627,497.181818,39.229627,573.4,1.949359,571.700000,2.311805,570.333333,2.742956,569.100000,3.291136,567.640000,4.192056,421.8,1.095445,423.400000,2.221111,426.133333,4.453998,430.450000,8.982293,435.760000,13.516286
C_ID_3d0044924f,4,1,0,2017-01,0.392913,659.0,46.669048,659.0,46.669048,659.000000,46.669048,659.000000,46.669048,659.000000,46.669048,659.0,46.669048,659.0,46.669048,659.000000,46.669048,659.000000,46.669048,659.000000,46.669048,746.2,1.788854,743.000000,4.242641,740.333333,5.177791,738.150000,5.993637,734.000000,10.279429,576.0,1.581139,579.300000,4.110961,581.600000,4.717748,583.450000,5.236260,585.160000,5.842944
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,736.6,8.384510,734.833333,8.658329,734.833333,8.658329,734.833333,8.658329,734.833333,8.658329,733.0,8.276473,734.833333,8.658329,734.833333,8.658329,734.833333,8.658329,734.833333,8.658329
C_ID_186d6a6901,4,3,0,2017-09,0.142495,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,484.0,0.707107,483.200000,1.032796,481.733333,2.548576,480.100000,3.768289,477.880000,5.688585,334.8,2.863564,338.300000,5.056349,342.400000,7.327054,349.200000,14.986310,357.720000,22.053949
C_ID_cdbd2c0db2,1,3,0,2017-11,-0.159749,346.2,20.813457,346.2,20.813457,346.200000,20.813457,346.200000,20.813457,346.200000,20.813457,346.2,20.813457,346.2,20.813457,346.200000,20.813457,346.200000,20.813457,346.200000,20.813457,437.4,1.673320,435.600000,2.221111,432.866667,4.453998,430.550000,5.633313,428.280000,6.858814,330.2,0.447214,331.500000,1.649916,333.066667,2.840188,335.950000,5.744334,338.600000,7.494442


In [6]:
fname = '175_l05'
ignore_features = ['first_active_month', 'card_id']

df_feat = train_test
df_feat.fillna(-1, inplace=True)

for col in df_feat.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('date_diff')):continue
#     if not(col.count('trans')):continue

    feature = df_feat[col].values.astype('float32')
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
#     utils.to_pkl_gzip(path = f'../features/2_second_valid/{fname}_{col}@', obj=feature)

### 132 date_diffの前半、後半における違いを特徴にする. 全体verも

In [7]:
from dateutil.parser import parse

# 日付の重複を削除するため
auth1['purchase_date'] =  auth1['purchase_date'].map(lambda x: str(x)[:10])
auth0['purchase_date'] =  auth0['purchase_date'].map(lambda x: str(x)[:10])

# 前回購買日との差分
def diff_shift_date(df, lag):
    
    # shiftは正が下方向、負が上方向にズレる
    df = df[[key, 'purchase_date']].drop_duplicates()
    df.sort_values(by=[key, 'purchase_date'], inplace=True)
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['shift1_date'] = df.groupby(key)['purchase_date'].shift(lag)
    df = df[[key, 'purchase_date', 'shift1_date']].dropna()
    df['date_diff_lag1'] = (df['purchase_date'] - df['shift1_date']).dt.days
    return df
    
for i in tqdm(range(1, 2, 1)):
    diff_auth1 = diff_shift_date(auth1, i)
    diff_auth0 = diff_shift_date(auth0, i)
    
display(diff_auth1.head())
display(diff_auth0.head())

100%|██████████| 1/1 [00:12<00:00, 12.93s/it]


,card_id,purchase_date,shift1_date,date_diff_lag1
19095792,C_ID_00007093c1,2017-02-16,2017-02-14,2
19095793,C_ID_00007093c1,2017-02-20,2017-02-16,4
19095848,C_ID_00007093c1,2017-03-03,2017-02-20,11
19095855,C_ID_00007093c1,2017-03-06,2017-03-03,3
19095847,C_ID_00007093c1,2017-03-08,2017-03-06,2


,card_id,purchase_date,shift1_date,date_diff_lag1
19095851,C_ID_00007093c1,2017-03-15,2017-03-11,4
19095860,C_ID_00007093c1,2017-04-19,2017-03-15,35
19095799,C_ID_00007093c1,2017-05-11,2017-04-19,22
19095795,C_ID_00007093c1,2017-05-18,2017-05-11,7
19095881,C_ID_00007093c1,2017-06-14,2017-05-18,27


In [8]:
df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')
train_test = pd.concat([df_train, df_test], axis=0)
train_test.set_index(key, inplace=True)


for diff, hist_new_name in zip([diff_auth1, diff_auth0], ['auth1', 'auth0']):
    df = diff[[key, 'purchase_date', 'date_diff_lag1']].drop_duplicates()
    
    aggs = {}
    aggs['date_diff_lag1'] = ['mean', 'std']

    df.sort_values(by=[key, 'purchase_date'], inplace=True)
    df = utils.row_number(df=df, level=key)
    
    for i in range(5, 20, 5):
        
        feat_n = f'{hist_new_name}_first'
        first = df.query(f"row_no<={i}")
        tmp = first.groupby(key)['date_diff_lag1'].agg(aggs)
        new_cols = get_new_columns(f'{feat_n}{i}', aggs)
        tmp.columns = new_cols
        train_test = train_test.join(tmp)
    
    
    df.sort_values(by=[key, 'purchase_date'], ascending=False, inplace=True)
    df = utils.row_number(df=df, level=key)
    for i in range(5, 20, 5):
         
        feat_n = f'{hist_new_name}_last'
        last = df.query(f"row_no<={i}")
        tmp = last.groupby(key)['date_diff_lag1'].agg(aggs)
        new_cols = get_new_columns(f'{feat_n}{i}', aggs)
        tmp.columns = new_cols
        train_test = train_test.join(tmp)

100%|██████████| 3/3 [00:00<00:00, 192.64it/s]


In [9]:
train_test.head()

,feature_1,feature_2,feature_3,first_active_month,target,auth1_first5_date_diff_lag1_mean,auth1_first5_date_diff_lag1_std,auth1_first10_date_diff_lag1_mean,auth1_first10_date_diff_lag1_std,auth1_first15_date_diff_lag1_mean,auth1_first15_date_diff_lag1_std,auth1_last5_date_diff_lag1_mean,auth1_last5_date_diff_lag1_std,auth1_last10_date_diff_lag1_mean,auth1_last10_date_diff_lag1_std,auth1_last15_date_diff_lag1_mean,auth1_last15_date_diff_lag1_std,auth0_first5_date_diff_lag1_mean,auth0_first5_date_diff_lag1_std,auth0_first10_date_diff_lag1_mean,auth0_first10_date_diff_lag1_std,auth0_first15_date_diff_lag1_mean,auth0_first15_date_diff_lag1_std,auth0_last5_date_diff_lag1_mean,auth0_last5_date_diff_lag1_std,auth0_last10_date_diff_lag1_mean,auth0_last10_date_diff_lag1_std,auth0_last15_date_diff_lag1_mean,auth0_last15_date_diff_lag1_std
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_92a2005557,5,2,1,2017-06,-0.820283,1.40,0.547723,1.30,0.483046,1.200000,0.414039,1.60,0.894427,1.80,1.032796,2.533333,2.099887,15.4,19.06043,11.3,14.750141,11.3,14.750141,7.2,9.176056,11.3,14.750141,11.3,14.750141
C_ID_3d0044924f,4,1,0,2017-01,0.392913,2.00,1.224745,2.20,1.032796,2.600000,1.404076,1.00,0.000000,1.30,0.674949,1.266667,0.593617,66.0,NaN,66.0,NaN,66.0,NaN,66.0,NaN,66.0,NaN,66.0,NaN
C_ID_d639edf6cd,2,2,0,2016-08,0.688056,4.25,5.852350,4.25,5.852350,4.250000,5.852350,4.25,5.852350,4.25,5.852350,4.250000,5.852350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_186d6a6901,4,3,0,2017-09,0.142495,1.60,0.894427,1.40,0.699206,1.466667,0.743223,3.00,1.581139,2.80,1.316561,3.800000,3.629246,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
C_ID_cdbd2c0db2,1,3,0,2017-11,-0.159749,1.00,0.000000,1.40,0.966092,1.333333,0.816497,1.00,0.000000,1.30,0.674949,1.466667,0.743223,38.0,NaN,38.0,NaN,38.0,NaN,38.0,NaN,38.0,NaN,38.0,NaN


In [10]:
fname = '176_l05'
ignore_features = ['first_active_month', 'card_id']

df_feat = train_test
df_feat.fillna(-1, inplace=True)

for col in df_feat.columns:
#     if col in ignore_features:
#         continue
#     if not(col.count('feature_')):continue
    if not(col.count('date_diff')):continue
#     if not(col.count('trans')):continue

    feature = df_feat[col].values.astype('float32')
    col = col.replace('date_diff', 'date_diff_lag1')
    utils.to_pkl_gzip(path = f'../features/1_first_valid/{fname}_{col}@', obj=feature)
#     utils.to_pkl_gzip(path = f'../features/2_second_valid/{fname}_{col}@', obj=feature)

In [54]:
# df = diff[[key, 'purchase_date', 'date_diff']].drop_duplicates()
# df_train = utils.read_df_pkl('../input/train0*')
# df_test = utils.read_df_pkl('../input/test0*')
# hist_new_name = 'new'
# aggs = {}
# aggs['date_diff'] = ['mean', 'std']

# df.sort_values(by=[key, 'purchase_date'], inplace=True)
# df = utils.row_number(df=df, level=key)
# tmp = df.groupby(key)['date_diff'].agg(aggs)
# new_cols = get_new_columns(f'{hist_new_name}', aggs)
# tmp.columns = new_cols
# df_train = df_train.merge(tmp.reset_index(), how='left', on=key)
# df_test = df_test.merge(tmp.reset_index(), how='left', on=key)
# df_train.head()


100%|██████████| 3/3 [00:00<00:00, 160.55it/s]

,first_active_month,card_id,feature_1,feature_2,feature_3,target,new_date_diff_mean,new_date_diff_std
0,2017-06,C_ID_92a2005557,5,2,1,-0.820283,2.750000,2.359193
1,2017-01,C_ID_3d0044924f,4,1,0,0.392913,19.000000,21.000000
2,2016-08,C_ID_d639edf6cd,2,2,0,0.688056,NaN,NaN
3,2017-09,C_ID_186d6a6901,4,3,0,0.142495,7.000000,5.966574
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159749,2.192308,1.720912
